## TF-IDF module
### => 문서 내부의 특정 단어가 어느정도 빈도로 있는지 파악하기 위함

In [9]:
# Library
from konlpy.tag import Okt
import pickle
import numpy as np

In [10]:
okt = Okt()

word_dic = {"_id" : 0} # 단어가 들어가는 사전
dt_dic = {} # 문장 전체에서의 단어 출현 횟수
files = [] # 문서들을 저장할 리스트

In [36]:
def tokenize(text):
    result = []
    
    word_s = okt.pos(text, norm = True, stem = True)
    
    for n, h in word_s:
        
        if not (h in ["Noun", "Verb", "Adjective"]):
            continue
        if h == "Punctuation" and h2 == "Number":
            continue
            
        result.append(n)
    
    return result

In [37]:
def words_to_ids(words, auto_add = True):
    result = []
    
    for w in words:
        
        if w in word_dic:
            result.append(word_dic[w])
            continue
        
        elif auto_add:
            id = word_dic[w] = word_dic["_id"]
            word_dic["_id"] +=1
            result.append(id)
            
    return result

In [38]:
def add_text(text):
    ids = words_to_ids(tokenize(text))
    files.append(ids)

In [39]:
def add_file(path):
    with open(path, "r", encoding = "utf-8") as f:
        s = f.read()
        
        add_text(s)

In [40]:
def calc_files():
    
    global dt_dic
    result = []
    doc_count = len(files)
    dt_dic = {}
    
    # 단어 출현 횟수 카운트
    
    for words in files:
        
        used_word = {}
        
        data = np.zeros(word_dic["_id"])
        
        for id in words:
            data[id] += 1
            
        # 단어 t가 사용되고 있을 경우 dt_dic의 수를 1 더하기
        for id in used_word:
                
            if not(id in dt_dic):
                dt_dic[id] = 0
            dt_dic[id] +=1
                
        # 정규화하기
        data = data / len(words)
        result.append(data)    
    
    # tf-idf 계산
    for i, doc in enumerate(result):
        
        for id, v in enumerate(doc):
            
            idf = np.log(doc_count / dt_dic[id]) + 1
            
            doc[id] = min([doc[id] * idf, 1.0])
            
        result[i] = doc
        
    return result

In [41]:
def save_dic(fname):
    
    pickle.dump([word_dic, dt_dic, files], open(fname, "wb"))

In [42]:
def load_dic(fname):
    
    global word_dic, dt_dic, files
    
    n = pickle.load(open(fname, "rb"))
    
    word_dic, dt_dic, files = n

In [43]:
def calc_text(text):
    
    data = np.zeros(word_dic["_id"])
    words = words_to_ids(tokenize(text), False)
    
    for w in words:
        
        data[w] += 1
        
    data = data / len(words)
    
    for id, v in enumerate(data):
        
        idf = np.log(len(files) / dt_dic[id] + 1)
        
        data[id] = min([data[id] * idf, 1.0])
    
    return data

In [44]:
if __name__ == "__main__":
    
    add_text("비")    
    add_text("오늘은 비가 내렸어요.")
    add_text("오늘은 더웠지만 오후부터 비가 내렸다.")
    add_text("비가 내리는 일요일이다.")
    print(calc_files())
    print(word_dic)

KeyError: 0